# Joint Course Jianning and Domenico 

## Course Details 

1. Course title and course code? 
2. 5 ECTS 
3. given in Q3
4. teaching by lectures and lab sessions
5. not all students same trajectory through course - how to accomodate in teaching triangle?  
7. assessment by ... 

## Block (1/3): Modeling Quasi-Electrostatic and Quasi-Magnetostatic Field (Week 1 and 2)

<b>Content</b>
1. Laplace and Poisson Equation for the Electrostatic Field in term of the electrical field potential. Material constitutive equations. Physical units;
2. electric energy density and magnetic energy density; 
3. (optional) Laplace and Poisson Equation for the Magnetostatic Field in terms of the scalar magnetic field potential. Material constitutive equations. Physical units;
4. double curl equation for the magnetic vector equation. Reduction to two-dimensional perpendicular current formulation;   
5. (optional): Reduction of double curl equation to to two-dimensional azimuthal current formulations; 
6. one-dimensional reference solutions: analytically, symbolically or built-in numerical tools;
7. two-dimensional reference solutions: analytically or built-in numerical tools;
8. Julia programminmg language   

<b>Provide Pointers to</b>
1. non-linear materials; 
2. eddy-current effects;
3. transient field computations; 

<b>Requires Input on</b>
1. material already available that can be reused (link here to courses in Q1 and Q2); 
2. how to compute inductance analytically or symbolically; 

## Block (2/3): (Rephrase to incorporate Jianning Material) One-Dimensional Finite Element Modeling (Week 3 and 4)

<b>Content</b>
1. Problem formulation: ordinary differential equation supplied with boundary value problems;
2. Weak formulation, variational formulation, weighted residuals;
3. Mesh generation, finite element basis functions; 
4. Assembly mass matrix, stiffness matrix, load vector;
5. Linear system solve for finite element solution;
6. Post-processing; 

<b>Provide Pointers to</b>

<b>Requires Input on</b>
1. exact problem formulation;
2. details on computations of the inductance $L$ either using energy or flux and preferably both; 

## Block (3/3): (Rephrase to incorporate Jianning Material) Two-Dimensional Finite Element Modeling (Week 5, 6 and 7) 

<b>Content</b>
1. extend above from 1D (only $x$) to 2D (both $x$ and $y$); 

<b>Provide Pointers to</b>

<b>Requires Input on</b>
1. pre-cooking geometry definition and mesh generation; 